## 1. Image Data Handling

In [1]:
%pylab inline

import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

import keras as K
from keras.models import Sequential
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, InputLayer, \
    Convolution2D, MaxPooling2D, Flatten,   \
    Dropout, BatchNormalization, GlobalAveragePooling2D

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
# To stop potential randomness
seed = 42
rng = np.random.RandomState(seed)

In [3]:
data = pd.read_csv('../datasets/emergency_classification/emergency_classification.csv')

In [4]:
data.head()

,image_names,emergency_or_not
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [5]:
data['emergency_or_not'].value_counts()

0    1361
1     991
Name: emergency_or_not, dtype: int64

In [6]:
# Let's have a look at random Emergency Vehicles

emergency_vehicles = data.loc[data['emergency_or_not']==1,"image_names"].tolist()

## 2. Data Loading and Preprocessing

In [7]:
images = []
for img_loc in data.image_names:
    img = image.load_img('../datasets/emergency_classification/images/' + img_loc)
    img = image.img_to_array(img)
    images.append(img)
    
images=np.array(images)

In [9]:
images.shape

(2352, 224, 224, 3)

In [10]:
train_x = images / images.max()
train_y = data.emergency_or_not.values

In [11]:
X_train, X_valid, Y_train, Y_valid=train_test_split(train_x,train_y,test_size=0.3, random_state=42)

## 3. Model Building

In [14]:
model = Sequential([
 InputLayer(input_shape=(224, 224, 3)),

 Convolution2D(32, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),

 Convolution2D(64, (5, 5), activation='relu', padding='same'),
 Convolution2D(64, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),

 Convolution2D(128, (5, 5), activation='relu', padding='same'),
 Convolution2D(128, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),
    
 Convolution2D(256, (5, 5), activation='relu', padding='same'),
 Convolution2D(256, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),
    
 Convolution2D(512, (5, 5), activation='relu', padding='same'),
 Convolution2D(512, (5, 5), activation='relu', padding='same'),

 GlobalAveragePooling2D(),
 BatchNormalization(),

 Dense(units=512, activation='sigmoid'),
 BatchNormalization(),
 Dropout(0.5),
    
 Dense(units=1, activation='sigmoid', name = 'preds'),
])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 224, 224, 32)      2432      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 112, 112, 64)      51264     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 112, 112, 64)      102464    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 56, 56, 64)        256       
__________

In [16]:
datagen = image.ImageDataGenerator(
    width_shift_range = 0.2,
    horizontal_flip = True
)

In [17]:
final_weights_path = '../models/best_cnn_model.h5'

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=20, verbose=1)
]

In [18]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [19]:
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=4),epochs=60,validation_data=(X_valid,Y_valid), callbacks=callbacks_list, shuffle=False, workers=5, use_multiprocessing=True)

Epoch 1/60
412/412 [==============================] - 27s 67ms/step - loss: 0.9213 - acc: 0.5267 - val_loss: 0.7031 - val_acc: 0.5722

Epoch 00001: val_loss improved from inf to 0.70315, saving model to ../models/best_cnn_model.h5
Epoch 2/60
412/412 [==============================] - 23s 56ms/step - loss: 0.8003 - acc: 0.5692 - val_loss: 0.7644 - val_acc: 0.4278

Epoch 00002: val_loss did not improve from 0.70315
Epoch 3/60
412/412 [==============================] - 23s 57ms/step - loss: 0.7585 - acc: 0.5868 - val_loss: 0.6845 - val_acc: 0.5567

Epoch 00003: val_loss improved from 0.70315 to 0.68448, saving model to ../models/best_cnn_model.h5
Epoch 4/60
412/412 [==============================] - 25s 60ms/step - loss: 0.7375 - acc: 0.5910 - val_loss: 0.7045 - val_acc: 0.5680

Epoch 00004: val_loss did not improve from 0.68448
Epoch 5/60
412/412 [==============================] - 25s 60ms/step - loss: 0.7127 - acc: 0.5922 - val_loss: 0.7224 - val_acc: 0.5623

Epoch 00005: val_loss did n

412/412 [==============================] - 25s 61ms/step - loss: 0.4723 - acc: 0.7755 - val_loss: 0.5109 - val_acc: 0.7663

Epoch 00042: val_loss did not improve from 0.40299
Epoch 43/60
412/412 [==============================] - 25s 61ms/step - loss: 0.4603 - acc: 0.7955 - val_loss: 0.4481 - val_acc: 0.8074

Epoch 00043: val_loss did not improve from 0.40299
Epoch 44/60
412/412 [==============================] - 25s 61ms/step - loss: 0.4743 - acc: 0.7816 - val_loss: 0.4467 - val_acc: 0.8258

Epoch 00044: val_loss did not improve from 0.40299
Epoch 45/60
412/412 [==============================] - 25s 61ms/step - loss: 0.4380 - acc: 0.8058 - val_loss: 0.4271 - val_acc: 0.8258

Epoch 00045: val_loss did not improve from 0.40299
Epoch 46/60
412/412 [==============================] - 25s 61ms/step - loss: 0.4590 - acc: 0.7828 - val_loss: 0.4522 - val_acc: 0.7861

Epoch 00046: val_loss did not improve from 0.40299
Epoch 47/60
412/412 [==============================] - 25s 61ms/step - loss: 

In [ ]:
model.load_weights(final_weights_path)

In [ ]:
predictions = model.predict_classes(X_valid).ravel()
prediction_probabilities = model.predict(X_valid).ravel()
all_vehicles = np.array(emergency_vehicles + nonemergency_vehicles)

_, valid_vehicles, _, valid_y = train_test_split(all_vehicles,train_y,test_size=0.3, random_state=42)

In [ ]:
correct_predictions = valid_vehicles[predictions == Y_valid]

index = rng.choice(range(len(correct_predictions)))
img_name = correct_predictions[index]


prob = (prediction_probabilities[predictions == Y_valid] * 100).astype(int)[index]

img = imread('../datasets/emergency_classification/images/' + img_name)

print(prob , '% sure that it is emergency')
pylab.imshow(img)
pylab.axis('off')
pylab.show()

In [ ]:
incorrect_predictions = valid_vehicles[predictions != Y_valid]
index = rng.choice(range(len(incorrect_predictions)))
img_name = incorrect_predictions[index]


prob = (prediction_probabilities[predictions != Y_valid] * 100).astype(int)[index]

img = imread('../datasets/emergency_classification/images/' + img_name)

print(prob , '% sure that it is emergency')
pylab.imshow(img)
pylab.axis('off')
pylab.show()

In [ ]:
print(classification_report(predictions, Y_valid))

In [ ]:
tn, fp, fn, tp = confusion_matrix(predictions, Y_valid).ravel()

In [ ]:
tn, tp, fn, fp

In [ ]:
roc_auc_score(predictions, Y_valid)

In [ ]:
top_layer = model.layers[0]
plt.imshow(top_layer.get_weights()[0][:, :, :, 0].squeeze())
plt.show()

In [ ]:
min_val = np.abs(top_layer.get_weights()[0][:, :, :, 0].squeeze().min())

In [ ]:
min_val

In [ ]:
plt.imshow(top_layer.get_weights()[0][:, :, :, 0].squeeze() + min_val)

In [ ]:
from vis.visualization import visualize_activation
from vis.utils import utils
from keras import activations

plt.rcParams['figure.figsize'] = (18, 6)

layer_idx = -1

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear

# This is the output node we want to maximize.
filter_idx = 0
img = visualize_activation(model, layer_idx, filter_indices=filter_idx)
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
from vis.visualization import get_num_filters

# The name of the layer we want to visualize
# You can see this in the model definition.
layer_idx = 4

# Visualize all filters in this layer.
filters = np.arange(get_num_filters(model.layers[layer_idx]))

# Generate input image for each filter.
vis_images = []
for idx in filters[:4]:
    img = visualize_activation(model, layer_idx, filter_indices=idx)
    
    # Utility to overlay text on image.
    img = utils.draw_text(img, 'Filter {}'.format(idx))    
    vis_images.append(img)

# Generate stitched image palette with 8 cols.
stitched = utils.stitch_images(vis_images, cols=8)    
plt.axis('off')
plt.imshow(stitched)
plt.title(layer_idx)
plt.show()

In [ ]:
img_name = rng.choice(emergency_vehicles)

img = imread('../datasets/emergency_classification/images/' +img_name)

In [ ]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam
from vis.visualization import visualize_saliency, overlay


plt.figure()
f, ax = plt.subplots()
plt.suptitle("vanilla")

grads = visualize_cam(model, -1, filter_indices=0, 
                      seed_input=img, backprop_modifier=None)        
# Lets overlay the heatmap onto original image.    
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
ax.imshow(overlay(jet_heatmap, img))

In [ ]:
for img_name in emergency_vehicles:
    img = imread('../datasets/emergency_classification/images/' + img_name)
    
    plt.figure()
    f, ax = plt.subplots()
    plt.suptitle("vanilla")

    grads = visualize_cam(model, -1, filter_indices=0, 
                          seed_input=img, backprop_modifier=None)        
    # Lets overlay the heatmap onto original image.    
    jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
    ax.imshow(overlay(jet_heatmap, img))
    plt.show()

In [ ]:
vehicles = glob('../datasets/emergency_classification/images/*.jpg')

In [ ]:
img_name = vehicles[285]
img = imread(img_name)

plt.figure()
f, ax = plt.subplots()
plt.suptitle("vanilla")

grads = visualize_cam(model, -1, filter_indices=0, 
                      seed_input=img, backprop_modifier=None)        
# Lets overlay the heatmap onto original image.    
jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
ax.imshow(overlay(jet_heatmap, img))
plt.show()